# Baseline MNIST

In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
import tensorflow.keras.datasets.mnist as mnist

In [3]:
import vbranch

In [4]:
save = True
model_id = 2

## Load Data

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [6]:
input_dim = 784
num_classes = 10

In [7]:
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes)

In [8]:
X_train.shape

(60000, 28, 28, 1)

## Build Model

In [9]:
BATCH_SIZE = 32
EPOCHS = 10
STEPS_PER_EPOCH = 100
model_path = './models/model_' + str(model_id)

In [10]:
tf.reset_default_graph()

train_data = (X_train.astype('float32'), y_train_one_hot)
test_data = (X_test.astype('float32'), y_test_one_hot)

batch_size = tf.placeholder('int64', name='batch_size')

train_dataset = tf.data.Dataset.from_tensor_slices(train_data).\
    batch(batch_size).repeat().\
    shuffle(buffer_size=4*BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices(test_data).\
    batch(batch_size).repeat()

iter_ = tf.data.Iterator.from_structure(train_dataset.output_types, 
                                       train_dataset.output_shapes)
inputs, labels_one_hot = iter_.get_next()

train_init_op = iter_.make_initializer(train_dataset)
test_init_op = iter_.make_initializer(test_dataset, name='test_init_op')

In [11]:
with tf.variable_scope('model_' + str(model_id)):
    model = vbranch.models.simple_cnn(inputs, num_classes, 16, 32)

In [12]:
model.summary()

i   Layer name                  Output shape          Parameters
----------------------------------------------------------------
    Input                  [None, 28, 28, 1]                    
----------------------------------------------------------------
0   conv2d_1_1            [None, 26, 26, 16]                 160
----------------------------------------------------------------
1   bn_1_1                [None, 26, 26, 16]                  32
----------------------------------------------------------------
2   relu_1_1              [None, 26, 26, 16]                   0
----------------------------------------------------------------
3   conv2d_1_2            [None, 24, 24, 16]                2320
----------------------------------------------------------------
4   bn_1_2                [None, 24, 24, 16]                  32
----------------------------------------------------------------
5   relu_1_2              [None, 24, 24, 16]                   0
-------------------------

In [13]:
tf.trainable_variables()

[<tf.Variable 'model_2/conv2d_1_1_f:0' shape=(3, 3, 1, 16) dtype=float32_ref>,
 <tf.Variable 'model_2/conv2d_1_1_b:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_2/bn_1_1_scale:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_2/bn_1_1_beta:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_2/conv2d_1_2_f:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
 <tf.Variable 'model_2/conv2d_1_2_b:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_2/bn_1_2_scale:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_2/bn_1_2_beta:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'model_2/conv2d_2_1_f:0' shape=(3, 3, 16, 32) dtype=float32_ref>,
 <tf.Variable 'model_2/conv2d_2_1_b:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'model_2/bn_2_1_scale:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'model_2/bn_2_1_beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'model_2/conv2d_2_2_f:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'model_2/conv2d_2_2_b:0' shape=

In [14]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels_one_hot, 
                                                                 logits=model.output), name='loss')
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [15]:
pred = tf.nn.softmax(model.output, name='pred')
pred_max = tf.one_hot(tf.argmax(pred, axis=-1), num_classes)
acc = tf.reduce_mean(tf.reduce_sum(labels_one_hot*pred_max, [1]), name='acc')

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for e in range(EPOCHS):
        print("Epoch {}/{}".format(e + 1, EPOCHS))
        progbar = tf.keras.utils.Progbar(STEPS_PER_EPOCH)
        
        sess.run(train_init_op, feed_dict={batch_size: BATCH_SIZE})

        for i in range(STEPS_PER_EPOCH):
            _, loss_value, acc_value = sess.run([train_op, loss, acc])
            
            if i == STEPS_PER_EPOCH - 1:
                sess.run(test_init_op, feed_dict={batch_size: len(X_test)})
                val_loss, val_acc = sess.run([loss, acc])
                progbar.update(i + 1, values=[("loss", loss_value), ("acc", acc_value), 
                                              ("val_loss", val_loss), ("val_acc", val_acc)])
            else:
                progbar.update(i + 1, values=[("loss", loss_value), ("acc", acc_value)])
    
    if save:
        saver = tf.train.Saver()
        path = os.path.join(model_path, 'ckpt')
        saver.save(sess, path)

Epoch 1/10
100/100 [==============================] - 33s 332ms/step - loss: 1.7310 - acc: 0.4822 - val_loss: 1.2768 - val_acc: 0.7807
Epoch 2/10
100/100 [==============================] - 14s 142ms/step - loss: 1.0870 - acc: 0.8166 - val_loss: 0.8354 - val_acc: 0.8821
Epoch 3/10
100/100 [==============================] - 17s 166ms/step - loss: 0.7428 - acc: 0.8766 - val_loss: 0.5581 - val_acc: 0.9214
Epoch 4/10
100/100 [==============================] - 16s 159ms/step - loss: 0.5453 - acc: 0.9006 - val_loss: 0.4000 - val_acc: 0.9420
Epoch 5/10
100/100 [==============================] - 15s 153ms/step - loss: 0.4102 - acc: 0.9263 - val_loss: 0.3087 - val_acc: 0.9532
Epoch 6/10
100/100 [==============================] - 15s 153ms/step - loss: 0.3310 - acc: 0.9394 - val_loss: 0.2581 - val_acc: 0.9534
Epoch 7/10
100/100 [==============================] - 18s 185ms/step - loss: 0.2766 - acc: 0.9481 - val_loss: 0.2425 - val_acc: 0.9526
Epoch 8/10
100/100 [==============================] - 1

## Model Ensemble

In [17]:
test_pred = []
test_loss = []
test_acc = []

num_models = 2
graphs = [tf.Graph() for _ in range(num_models)]
sessions = [tf.Session(graph=g) for g in graphs]

for i in range(len(graphs)):
    with graphs[i].as_default():
        model_path = './models/model_' + str(i + 1)
        meta_path = os.path.join(model_path, 'ckpt.meta')
        ckpt = tf.train.get_checkpoint_state(model_path)
        
        imported_graph = tf.train.import_meta_graph(meta_path)
        imported_graph.restore(sessions[i], ckpt.model_checkpoint_path)
                
        sessions[i].run('test_init_op', feed_dict={'batch_size:0': len(X_test)})
        
        pred, loss, acc = sessions[i].run(['pred:0', 'loss:0', 'acc:0'])
        test_pred.append(pred)
        test_loss.append(loss)
        test_acc.append(acc)

INFO:tensorflow:Restoring parameters from ./models/model_1/ckpt
INFO:tensorflow:Restoring parameters from ./models/model_2/ckpt


In [18]:
test_acc

[0.9671, 0.9675]

In [19]:
def compute_acc(pred, labels_one_hot):
    pred_max = tf.keras.utils.to_categorical(np.argmax(pred, axis=-1), num_classes)
    return np.mean(np.sum(labels_one_hot*pred_max, axis=1))

In [20]:
for i in range(num_models):
    assert compute_acc(test_pred[i], y_test_one_hot) == test_acc[i]

In [21]:
pred_avg = np.mean(test_pred, axis=0)
print(compute_acc(pred_avg, y_test_one_hot))

0.972
